In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

In [ ]:
def apply_offset(x, y, offset=0):
    x = np.append(x, np.zeros(abs(offset)))
    y = np.append(np.zeros(abs(offset)), y)
    return x, y

def clean(a, b, offset=None):
    min_a = np.min(a[:,1])
    min_b = np.min(b[:,1])
    a = a[:,1] - min_a
    b = b[:,1] - min_b

    if offset is not None:
        a, b = apply_offset(a, b, offset=offset)
        
    if len(a) < len(b):
        b = b[:len(a)]
    elif len(b) < len(a):
        a = a[:len(b)]
    
    a_auc = np.trapz(a, dx=1)
    b_auc = np.trapz(b, dx=1)

    a = a/a_auc
    b = b/b_auc

    return a,b


In [ ]:
def find_offset(a, b):
    diff = b - a
    max_pos = list(diff).index(max(diff))
    max_pos_a = list(a).index(max(a))
    offset = max_pos_a - max_pos
    return offset


def R_sq(y, y_fit):
    ss_res = np.sum((y - y_fit) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)

    r2 = 1 - (ss_res / ss_tot)
    
    return 1-abs(r2)


def try_rel_fluor_val(scale_factor, g, r, _offset, get_func=False):
    z = np.zeros(_offset)

    _g_left = g[_offset:]
    _g_left = np.append(_g_left, [z])  

    fit = scale_factor * _g_left + g
    fit = fit/(1+k)
    
    if get_func:
        return R_sq(r, fit), fit
    else:
        return R_sq(r, fit)
    

In [ ]:
from numpy import genfromtxt

def compress(arr, skip_every_n):
    skip_every_n = np.floor(skip_every_n)
    outarr = [x for (n, x) in enumerate(arr) if bool(n % skip_every_n)]
    #print('{0} -> delta len change {1}'.format(skip_every_n, (len(arr) - len(outarr))/len(arr) ))
    outarr = np.append(outarr, [np.zeros(len(arr) - len(outarr))])
    
    return outarr

def try_compress(compress_fold_factor, scale_factor, purified, copurified, _offset=0,
                get_func=False):
    if compress_fold_factor < 1:
        return 1E6
    
    z = np.zeros(_offset)

    _g_left = copurified[_offset:]
    _g_left = np.append(_g_left, [z])  

    monoligated = scale_factor * _g_left
    
    skip_every_n = np.floor(compress_fold_factor)
    #print("skip_every_n: ", skip_every_n)
    monoligated = compress(monoligated, skip_every_n)
    
    fit = monoligated + copurified
    fit = fit/np.trapz(fit, dx=1)
    
    if get_func:
        return R_sq(purified, fit), fit
    else:
        return R_sq(purified, fit)

def _try_compress(x, purified, copurified, _offset=0,
                get_func=False):
    return try_compress(x[0], x[1], purified, copurified, _offset=0,
                get_func=False)

def fit_and_plot(purified, copurified):
    """
    The purified and copurified arguments are tables of (x,y) values that form curves.
    The copurified curve should always start jumping in signal to the right of the
    purified curve."""
    copurified, purified = clean(copurified, purified, offset=0)

    offset = find_offset(copurified, purified)
    z = np.zeros(offset)

    
    scaling_results = sp.optimize.minimize_scalar(try_rel_fluor_val, args=(copurified, purified, offset))
    print(scaling_results)
    scale_factor = scaling_results.x
    
    res = sp.optimize.minimize_scalar(try_compress, args=(scale_factor, purified, copurified, offset))
    print('compression fit:')
    print(res)

    print('----\nTrying 2D optimization...')
    res2d = minimize(_try_compress, [float(res.x), scale_factor], args=(purified, copurified, offset),
                    #method='Nelder-Mead',
                    bounds=[(0, 300),(0.05, 10)])
                     
    print(res2d)
    print('Finished trying 2D optimization (not using results).\n' + '-'*10,)

    
    if scaling_results.fun > res.fun:
        print("Curve compression helped the fit.")
        (r2, fit) = try_compress(res.x, scale_factor, purified, copurified, offset, get_func=True)
    #(r2, fit) = try_rel_fluor_val(scale_factor, copurified, purified, offset, get_func=True)
    else:
        (r2, fit) = try_rel_fluor_val(scale_factor, copurified, purified, offset, get_func=True)
    print("\nThe copurified curve is {0:.4f} of the total ({0:.4f} efficiency). R**2: {1:.4f}".format(
        1./(1.+scale_factor), 1-r2))
    
    xaxis = range(len(purified))
    plt.plot(xaxis, purified, c='r')
    _g = copurified[offset:]
    _g = np.append(_g, [z])
    plt.plot(xaxis, copurified , c='g')
    plt.plot(xaxis, fit, c='k')
    
    monoligated = copurified[offset:]
    monoligated = np.append(monoligated, [z])
    monoligated = scale_factor * monoligated
    combined = monoligated + copurified
    _auc = np.trapz(combined, dx=1)
    monoligated = monoligated/_auc
    plt.plot(xaxis,  monoligated, c='gray', linestyle=':')
    
    plt.show()
    plt.clf()
    
    return 1./(1.+res.x)


a = genfromtxt('/Users/dfporter/Desktop/800L5IP.csv', delimiter=',')
b = genfromtxt('/Users/dfporter/Desktop/700L5IP.csv', delimiter=',')
eff_l3 =  fit_and_plot(b, a)
a = genfromtxt('/Users/dfporter/Desktop/L3ip800.csv', delimiter=',')
b = genfromtxt('/Users/dfporter/Desktop/L3IP_700.csv', delimiter=',')
eff_l5 = fit_and_plot(a, b)
print('L5/L3 ligation efficiency: {0:.5f}'.format(eff_l5/eff_l3))

In [ ]:
import scipy.stats as ss
import scipy.optimize as so

#define a likelihood function
def likelihood_f(P, x, neg=1):
    n=np.round(P[0]) #by definition, it should be an integer 
    p=P[1]
    loc=np.round(P[2])

    # Take the log of each value in the pmf (the prob of each obs) and sum to get the
    # probability of all observations.
    return neg*(np.log(ss.nbinom.pmf(x, n, p, loc))).sum()

#generate a random variable
X=ss.nbinom.rvs(n=100, p=0.4, loc=0, size=1000)
#print(X)
#The likelihood
like = likelihood_f([100,0.4,0], X)
print(like)

result=so.fmin(likelihood_f, [50, 0.5, 0], args=(X,-1), full_output=True, disp=False)
P1=result[0]
print(result)

#a = np.random.negative_binomial(10, 0.5, 1000)
X=ss.nbinom.rvs(n=1, p=0.0002, loc=0, size=10000)
plt.hist(X, normed=True)
plt.plot(range(50000), ss.nbinom.pmf(range(50000), np.round(1), 0.0002, 0), 'g-')
plt.show()
plt.clf()
